In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 🔹 Dummy Wallet Data (yeh manually daal diya gaya hai)
data = {
    "wallet_id": [
        "0xfaa0768bde629806739c3a4620656c5d26f44ef2",
        "0xa0ee7a142d267c1f36714e4a8f75612f20a79720",
        "0x742d35cc6634c0532925a3b844bc454e4438f44e"
    ],
    "total_borrow": [10000, 5000, 2000],           # in USD
    "total_supply": [15000, 10000, 1000],          # in USD
    "repayment_ratio": [0.9, 0.4, 0.8],            # repaid / borrowed
    "liquidation_count": [1, 3, 0],                # number of times liquidated
    "days_since_last_txn": [10, 100, 30],          # activity recency
    "unique_assets": [5, 2, 3]                     # no. of assets interacted with
}

df = pd.DataFrame(data)
df.head()

,wallet_id,total_borrow,total_supply,repayment_ratio,liquidation_count,days_since_last_txn,unique_assets
0,0xfaa0768bde629806739c3a4620656c5d26f44ef2,10000,15000,0.9,1,10,5
1,0xa0ee7a142d267c1f36714e4a8f75612f20a79720,5000,10000,0.4,3,100,2
2,0x742d35cc6634c0532925a3b844bc454e4438f44e,2000,1000,0.8,0,30,3


In [6]:
#Risk Feature: Borrow/Supply Ratio
df["borrow_supply_ratio"] = df["total_borrow"] / df["total_supply"]

#Risk Feature Columns
features = [
    "borrow_supply_ratio",
    "repayment_ratio",
    "liquidation_count",
    "days_since_last_txn",
    "unique_assets"
]

# Normalize
scaler = MinMaxScaler()
normalized = scaler.fit_transform(df[features])

# Invert positive indicators (repayment_ratio, unique_assets)
normalized[:, features.index("repayment_ratio")] = 1 - normalized[:, features.index("repayment_ratio")]
normalized[:, features.index("unique_assets")] = 1 - normalized[:, features.index("unique_assets")]

# Define weights (change as needed)
weights = np.array([
    0.3,  # borrow_supply_ratio
    0.25, # repayment_ratio
    0.2,  # liquidation_count
    0.15, # days_since_last_txn
    0.1   # unique_assets
])

# Risk Score Calculation
risk_scores = normalized @ weights
df["score"] = (1 - risk_scores) * 1000
df["score"] = df["score"].round().astype(int)

df[["wallet_id", "score"]]

,wallet_id,score
0,0xfaa0768bde629806739c3a4620656c5d26f44ef2,900
1,0xa0ee7a142d267c1f36714e4a8f75612f20a79720,300
2,0x742d35cc6634c0532925a3b844bc454e4438f44e,550


In [8]:
df[["wallet_id", "score"]]

,wallet_id,score
0,0xfaa0768bde629806739c3a4620656c5d26f44ef2,900
1,0xa0ee7a142d267c1f36714e4a8f75612f20a79720,300
2,0x742d35cc6634c0532925a3b844bc454e4438f44e,550


In [10]:
df[["wallet_id", "score"]].to_csv("wallet_scores.csv", index=False)
print("CSV file 'wallet_scores.csv' saved successfully.")

CSV file 'wallet_scores.csv' saved successfully.


In [ ]:
Explanation

**1. Data Collection:**
Dummy data manually input kiya gaya. Real-world mein Compound V2 ka data `The Graph` se fetch kiya ja sakta hai.

**2. Feature Selection:**
- `borrow/supply ratio`: Zyada leverage toh zyada risk
- `repayment_ratio`: Low value means repayment poor
- `liquidation_count`: More = bad
- `days_since_last_txn`: Old wallet ho toh risky ya inactive
- `unique_assets`: Diversification = less risk

**3. Scoring:**
- Features normalize kiye (0–1)
- Positive indicators (repayment ratio, assets) ko invert kiya
- Weighted sum banaya
- Score = (1 - risk) * 1000

**4. Justification:**
Wallets jinme zyada borrowing, kam repayment ya liquidation history ho — unka risk zyada hai. Isi basis pe score diya gaya.